In [3]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    # Python is 0-indexed, so we use start_week - 1
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "B1"

# Existing + New Submachines
submachines_data = [
    # 1. Cracker Mill
    {
        "sub_name": "طاحونة الكراكر",
        "tasks": [
            ("التأكد من عمل سير التغذية بكفاءة",                    300, 15,  "1", 1),
            ("معالجة أى تسريب زيت من الجيربوكس",                    310, 45,  "1", 1),
            ("ضبط قطع سكينة سير التغذية بكفاءة",                    320, 45,  "1", 4),
            ("تغيير سكينتى سير التغذية (الروتارى و الكونتر)",       330, 120, "2", 8),
            ("تنظيف رولات السير الناقل من الكراكر",                 340, 60,  "1", 1),
            ("تنظيف السير الناقل من الكراكر",                       350, 60,  "1", 8),
            ("معالجة أى تسريب مياه من الروتارى جوينت",              360, 60,  "2", 1)
        ]
    },
    # 2. Pre-feeder Mill
    {
        "sub_name": "طاحونة البريفيدر",
        "tasks": [
            ("معالجة اى شحط مطاط فى بلتة الأستوك بلندر",            370, 45,  "2", 1),
            ("تنظيف رولات السير الناقل من البريفيدر",               340, 60,  "1", 8),
            ("تنظيف السير الناقل من البريفيدر",                     350, 60,  "1", 8),
            ("التأكد من قطع السكاكين السفلية",                      380, 10,  "1", 4),
            ("سن سكاكين القطع السفلية",                             390, 30,  "1", 4),
            ("معالجة أى تسريب مياه من الروتارى جوينت",              360, 60,  "2", 1)
        ]
    },
    # 3. Feeder Mill
    {
        "sub_name": "طاحونة الفيدر",
        "tasks": [
            ("معالجة اى شحط مطاط فى بلتة الأستوك بلندر",            370, 45,  "1", 1),
            ("تنظيف رولات السير الناقل العلوى من الفيدر",           340, 60,  "1", 8),
            ("تنظيف السير الناقل العلوى من الفيدر",                 350, 60,  "1", 8),
            ("تنظيف رولات السير الناقل السفلى من الفيدر",           340, 60,  "1", 8),
            ("تنظيف السير الناقل السفلى من الفيدر",                 350, 60,  "1", 8),
            ("التأكد من قطع السكاكين السفلية",                      380, 15,  "1", 4),
            ("سن سكاكين القطع السفلية",                             390, 30,  "1", 4),
            ("معالجة أى تسريب مياه من الروتارى جوينت",              360, 60,  "2", 1)
        ]
    },
    # 4. Calender (الكاندر)
    {
        "sub_name": "الكاندر",
        "tasks": [
            ("تنظيف رولات سلندرات الكالندر الاربعة",                340, 60,  "1", 1),
            ("ضبط حركة سير الويج-واج العلوى",                       300, 30,  "1", 4),
            ("تنظيف رولات وسير الويج-واج العلوى",                   340, 60,  "1", 8),
            ("ضبط حركة سير الويج-واج السفلى",                       300, 30,  "1", 4),
            ("معالجة اى تسريب زيت من وحدة هيدروليك الكالندر",       310, 60,  "1", 8),
            ("تنظيف فلاتر زيت وحدة هيدروليك الكالندر",              400, 60,  "1", 13),
            ("التأكد من ضغوط زيت الهيدروليك",                       410, 30,  "1", 1),
            ("معالجة اى تسريب زيت من وحدة تزييت الكالندر",          310, 60,  "1", 8),
            ("تنظيف فلاتر زيت وحدة تزييت الكالندر",                 400, 60,  "1", 1),
            ("التأكد من ضغوط زيت وحدة التزييت",                     410, 30,  "1", 8),
            ("التأكد من عمل بساتم البري-لودينج بكفاءة",             420, 60,  "1", 13),
            ("معالجة أي تسريب مياه من الروتارى جوينت",              360, 60,  "2", 8),
            ("ضبط البنشر السفلى",                                   380, 30,  "1", 8),
            ("تغيير سكاكين البنشر السفلى",                          390, 60,  "1", 26),
            ("ضبط البنشر العلوى",                                   380, 30,  "1", 8),
            ("تغيير سكاكين البنشر السفلى",                          390, 60,  "1", 26), # Note: Text says Lower in image 37 as well
            ("ضبط الخلوص بين الدرافيل",                             430, 180, "2", 26),
            ("ضبط سكاكين قطع العرض",                                380, 45,  "1", 13),
            ("تغيير سكاكين قطع العرض",                              390, 45,  "1", 26)
        ]
    },
    # 5. Wire Room (حجرة السلك)
    {
        "sub_name": "حجرة السلك",
        "tasks": [
            ("تغيير بلى شبابيك السلك الصغيرة",                      440, 90,  "1", 52),
            ("اتمام نقص جلب البورسيلين فى شبابيك السلك",            440, 45,  "1", 26),
            ("التأكد من عمل فرامل بكرات الصف A بكفاءة",             450, 60,  "1", 1),
            ("تغيير ما يلزم فى فرامل بكرات الصف A",                 460, 120, "1", 13),
            ("التأكد من عمل فرامل بكرات الصف B بكفاءة",             450, 60,  "1", 1),
            ("تغيير ما يلزم فى فرامل بكرات الصف B",                 460, 120, "1", 13),
            ("التأكد من عمل فرامل بكرات الصف C بكفاءة",             450, 60,  "1", 1),
            ("تغيير ما يلزم فى فرامل بكرات الصف C",                 460, 120, "1", 13),
            ("التأكد من عمل فرامل بكرات الصف D بكفاءة",             450, 60,  "1", 1),
            ("تغيير ما يلزم فى فرامل بكرات الصف D",                 460, 120, "1", 13)
        ]
    },
    # 6. Rest of the Line (باقي الخط)
    {
        "sub_name": "باقي الخط",
        "tasks": [
            ("ضبط رولات ترابيزة المغناطيس",                         440, 45,  "1", 26),
            ("ضبط رول شد السلك",                                    440, 60,  "1", 13),
            ("التأكد من عمل الثرموريجيوليشن",                       470, 45,  "1", 8),
            ("معالجة أى تسريب مياه من الثرموريجيوليشن",             360, 60,  "1", 8),
            ("التاكد من عمل وحدة السنترة بكفاءة",                   420, 90,  "1", 1),
            ("ضبط ما يلزم وحدة السنترة",                            480, 90,  "1", 13),
            ("ضبط قطع سكينة الوايند-أب",                            320, 45,  "1", 13),
            ("تغيير سكينتى الوايند-أب (الروتارى و الكونتر)",        330, 120, "2", 26)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        # Default behavior
        start_week = 1
        step = period 
        
        # Special User Rules:
        if period == 26:
            start_week = 25  # Freq 26 starts W25
        elif period == 13:
            start_week = 1   
            step = 12        # "Starts W1 then next is W13" (Difference is 12)
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags with the calculated step and start_week
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'B1_Mills_PM_Plan_Complete.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\B1_Mills_PM_Plan_Complete.csv
Total Tasks Processed: 58


In [4]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "B1 Lubricate"

submachines_data = [
    # Image 1
    {
        "sub_name": "سير تغذية الكراكر",
        "tasks": [
            ("تنظيف", 500, 60, "1", 1),
            ("زيت الهواء", 510, 30, "1", 8),
            ("تشحيم", 520, 60, "1", 13),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    {
        "sub_name": "طاحونة الكراكر",
        "tasks": [
            ("تنظيف", 500, 60, "1", 1),
            ("دورة التشحيم", 540, 90, "1", 26),
            ("تشحيم", 520, 60, "1", 13),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    {
        "sub_name": "السير الناقل من الكراكر",
        "tasks": [
            ("تشحيم", 520, 60, "1", 13),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    {
        "sub_name": "طاحونة البريفيدر",
        "tasks": [
            ("تنظيف", 500, 60, "1", 1),
            ("دورة التشحيم", 540, 90, "1", 26),
            ("تشحيم", 520, 60, "1", 13),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    {
        "sub_name": "استوك بلندر البريفيدر",
        "tasks": [
            ("تزييت", 530, 90, "1", 52),
            ("تشحيم", 520, 60, "1", 13)
        ]
    },
    {
        "sub_name": "السير الناقل من البريفيدر",
        "tasks": [
            ("تشحيم", 520, 60, "1", 13),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    # Image 2
    {
        "sub_name": "طاحونة الفيدر",
        "tasks": [
            ("تنظيف", 500, 60, "1", 1),
            ("دورة التشحيم", 540, 90, "1", 26),
            ("تشحيم", 520, 60, "1", 13),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    {
        "sub_name": "استوك بلندر الفيدر",
        "tasks": [
            ("تزييت", 530, 90, "1", 52),
            ("تشحيم", 520, 60, "1", 13)
        ]
    },
    {
        "sub_name": "حجرة السلك",
        "tasks": [
            ("زيت الهواء", 510, 30, "1", 8),
            ("تشحيم", 520, 60, "1", 13)
        ]
    },
    {
        "sub_name": "خط الانتاج",
        "tasks": [
            ("زيت الهواء", 510, 30, "1", 8),
            ("تنظيف", 500, 60, "1", 8),
            ("تشحيم", 520, 60, "1", 13)
        ]
    },
    {
        "sub_name": "ترابيزة المغناطيس",
        "tasks": [
            ("زيت الهواء", 510, 30, "1", 8),
            ("تشحيم", 520, 60, "1", 13)
        ]
    },
    {
        "sub_name": "الكالندر",
        "tasks": [
            ("زيت الهواء", 510, 30, "1", 8),
            ("تشحيم", 520, 60, "1", 13),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    # Image 3
    {
        "sub_name": "وحدة هيدروليك الكالندر",
        "tasks": [
            ("تنظيف", 500, 60, "1", 1),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    {
        "sub_name": "وحدة تزييت الكالندر",
        "tasks": [
            ("تنظيف", 500, 60, "1", 1),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سيور تغذية الكالندر",
        "tasks": [
            ("تشحيم", 520, 60, "1", 13),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سيور الويج-واج",
        "tasks": [
            ("تشحيم", 520, 60, "1", 13),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    {
        "sub_name": "اسطوانات التبريد",
        "tasks": [
            ("تزييت", 530, 90, "1", 52),
            ("تشحيم", 520, 60, "1", 13)
        ]
    },
    {
        "sub_name": "الفستونـة",
        "tasks": [
            ("تزييت", 530, 90, "1", 52),
            ("تشحيم", 520, 60, "1", 13)
        ]
    },
    {
        "sub_name": "وحدة السنترة",
        "tasks": [
            ("تنظيف", 500, 60, "1", 1),
            ("تزييت", 530, 90, "1", 52)
        ]
    },
    {
        "sub_name": "الوايند-اب",
        "tasks": [
            ("تشحيم", 520, 60, "1", 13),
            ("تزييت", 530, 90, "1", 52)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        # Apply special frequency rules
        if period == 26:
            start_week = 25
        elif period == 13:
            start_week = 1   
            step = 12  # As per previous rule: starts W1 then next is W13
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'B1_Lubrication_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\B1_Lubrication_Plan.csv
Total Tasks Processed: 50


In [5]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---

all_machines_data = [
    # ---------------------------------------------------------
    # MACHINE 1: B3 0.BELT (No number implies 1)
    # ---------------------------------------------------------
    {
        "machine_name": "B3 0.BELT MECHANICAL",
        "submachines": [
            {
                "sub_name": "حجرة السلك",
                "tasks": [
                    ("تغيير بلى شبابيك السلك الصغيرة",              440, 90,  "1", 52),
                    ("التأكد من عمل فرامل بكرات الصف A بكفاءة",     450, 60,  "1", 13),
                    ("تغيير ما يلزم فى فرامل بكرات الصف A",         460, 120, "1", 13),
                    ("التأكد من عمل فرامل بكرات الصف B بكفاءة",     450, 60,  "1", 13),
                    ("تغيير ما يلزم فى فرامل بكرات الصف B",         460, 120, "1", 13),
                ]
            },
            {
                "sub_name": "الاكسترودر",
                "tasks": [
                    ("معالجة اى تسريب زيت من الجيربوكس",            310, 60,  "2", 26),
                    ("معالجة أى تسريب مياه من روتارى جوينت الاكسترودر", 360, 60,  "1", 13),
                    ("التأكد من عمل Driving Coupling بكفاءة",       380, 45,  "1", 13),
                    ("التأكد من عمل الثرموريجيوليشن",               470, 60,  "2", 8),
                    ("معالجة أى تسريب مياه من الثرموريجيوليشن",     360, 60,  "1", 8),
                ]
            }
        ]
    },
    {
        "machine_name": "B3 0.BELT LUBRICATION",
        "submachines": [
            {
                "sub_name": "خط تصنيع الزيرو",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تنظيف",      600, 60, "1", 8),
                    ("تشحيم",      620, 60, "1", 13),
                ]
            },
            {
                "sub_name": "الاكسترودر",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52),
                ]
            },
            {
                "sub_name": "درامات التبريد",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52),
                ]
            },
            {
                "sub_name": "الفستونـة",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52),
                ]
            },
            {
                "sub_name": "الوايندر",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13),
                ]
            }
        ]
    },
    # ---------------------------------------------------------
    # MACHINE 2: B3 0.BELT.2 (Explicitly numbered)
    # ---------------------------------------------------------
    {
        "machine_name": "B3 0.BELT.2 MECHANICAL",
        "submachines": [
            {
                "sub_name": "حجرة السلك",
                "tasks": [
                    ("تغيير بلى شبابيك السلك الصغيرة",              440, 90,  "1", 52),
                    ("التأكد من عمل فرامل بكرات الصف A بكفاءة",     450, 60,  "1", 13),
                    ("تغيير ما يلزم فى فرامل بكرات الصف A",         460, 120, "1", 13),
                    ("التأكد من عمل فرامل بكرات الصف B بكفاءة",     450, 60,  "1", 13),
                    ("تغيير ما يلزم فى فرامل بكرات الصف B",         460, 120, "1", 13),
                ]
            },
            {
                "sub_name": "الاكسترودر",
                "tasks": [
                    ("معالجة اى تسريب زيت من الجيربوكس",            310, 60,  "2", 26),
                    ("معالجة أى تسريب مياه من روتارى جوينت الاكسترودر", 360, 60,  "1", 13),
                    ("التأكد من عمل Driving Coupling بكفاءة",       380, 45,  "1", 13),
                    ("التأكد من عمل الثرموريجيوليشن",               470, 60,  "2", 8),
                    ("معالجة أى تسريب مياه من الثرموريجيوليشن",     360, 60,  "1", 8),
                ]
            }
        ]
    },
    {
        "machine_name": "B3 0.BELT.2 LUBRICATION",
        "submachines": [
            {
                "sub_name": "خط تصنيع الزيرو",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تنظيف",      600, 60, "1", 8),
                    ("تشحيم",      620, 60, "1", 13),
                ]
            },
            {
                "sub_name": "الاكسترودر",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52),
                ]
            },
            {
                "sub_name": "درامات التبريد",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52),
                ]
            },
            {
                "sub_name": "الفستونـة",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52),
                ]
            },
            {
                "sub_name": "الوايندر",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13),
                ]
            }
        ]
    }
]

data_rows = []

for machine in all_machines_data:
    mach_name = machine["machine_name"]
    global_task_counter = 1  # Reset counter for each machine
    
    for sub in machine["submachines"]:
        sub_name = sub["sub_name"]
        
        for task in sub["tasks"]:
            desc, code, mins, crew, period = task
            
            # --- Logic for Start Week and Step ---
            start_week = 1
            step = period 
            
            if period == 26:
                start_week = 25
            elif period == 13:
                start_week = 1   
                step = 12  # Custom step for 13 weeks
                
            row = {
                'Machine': mach_name,
                'Sub_Machine': sub_name, 
                '.': global_task_counter,
                'Task_Description': desc,
                'Task_Code': code,
                'Estimated_mins': mins,
                'Crew': crew,
                'Period_freq': period
            }
            
            # Apply week flags
            week_flags = get_weeks(step, start_week)
            for w_idx, flag in enumerate(week_flags, 1):
                row[f'W{w_idx}'] = flag
                
            data_rows.append(row)
            global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'B3_0_BELT_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {len(data_rows)}")

File saved successfully at: tabel\B3_0_BELT_PM_Plan.csv
Total Tasks Processed: 44


In [7]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "Roller head Lubrication"

submachines_data = [
    # --- Image 1 ---
    {
        "sub_name": '6" Ext. Feed. Conveyors',
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Liner Line",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تنظيف",      600, 60, "1", 8),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Extruder Zone",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Ext. Head Hyd. Unit",
        "tasks": [
            ("تنظيف",      600, 60, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Feed Roll Lub. Unit",
        "tasks": [
            ("تنظيف",      600, 60, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "2 - Roll Calender",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    # --- Image 2 ---
    {
        "sub_name": "Calender Lub. Unit",
        "tasks": [
            ("تنظيف",      600, 30, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Cal. Hyd. Unit",
        "tasks": [
            ("تنظيف",      600, 60, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Master Conveyor",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Cooling Drums",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Under Belt Conveyor",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Coupling Conveyor",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Centering Conveyors",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    # --- Image 3 ---
    {
        "sub_name": "Over - Head Conveyor",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Let-Off Cars",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Let-Off Conveyors",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Let-Off Festoon",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Cutting Festoon",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Cutting M/C",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Wind-Up Conveyors",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Wind-Up Cars",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        if period == 26:
            start_week = 25
        elif period == 13:
            start_week = 1   
            step = 12  # Custom step for 13 weeks
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'Roller_Head_Lubrication_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\Roller_Head_Lubrication_PM_Plan.csv
Total Tasks Processed: 47


In [8]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "Roller Head Mechanical"

submachines_data = [
    # Image 1
    {
        "sub_name": "سير التغذية",
        "tasks": [
            ("التأكد من عمل سير التغذية بكفاءة",                    300, 15,  "1", 13),
            ("تنظيف رولات سير التغذية",                             340, 60,  "1", 8),
            ("تنظيف سير التغذية",                                   350, 60,  "1", 8)
        ]
    },
    {
        "sub_name": "الاكسترودر",
        "tasks": [
            ("معالجة اى تسريب زيت من وحدة الهيدروليك",              310, 60,  "1", 8),
            ("معالجة اى تسريب زيت من الجيربوكس",                    310, 60,  "2", 8),
            ("معالجة اى تسريب زيت من الفيدر",                       310, 60,  "1", 8),
            ("تغيير بلى الفيدر",                                    440, 120, "2", 13),
            ("معالجة أى تسريب مياه من روتارى جوينت الاكسترودر",     360, 60,  "2", 8),
            ("التأكد من عمل سيور Driving بكفاءة",                   350, 60,  "1", 13),
            ("التأكد من عمل الثرموريجيوليشن",                       470, 45,  "1", 8),
            ("معالجة أى تسريب مياه من الثرموريجيوليشن",             360, 60,  "2", 8)
        ]
    },
    {
        "sub_name": "الكالندر",
        "tasks": [
            ("معالجة اى تسريب زيت من وحدة هيدروليك الكالندر",       310, 60,  "1", 8),
            ("التأكد من ضغوط زيت الهيدروليك",                       410, 30,  "1", 8),
            ("معالجة اى تسريب زيت من وحدة تزييت الكالندر",          310, 60,  "1", 8),
            ("تنظيف فلاتر زيت وحدة تزييت الكالندر",                 400, 60,  "1", 13),
            ("التأكد من ضغوط زيت وحدة التزييت",                     410, 30,  "1", 4),
            ("التأكد من عمل البساتم بكفاءة",                        420, 60,  "1", 13),
            ("معالجة أى تسريب مياه من الروتارى جوينت OP",           360, 60,  "2", 13),
            ("ضبط الخلوص بين الدرافيل",                             430, 180, "2", 26),
            ("ضبط سكاكين قطع العرض",                                380, 45,  "1", 13),
            ("تغيير سكاكين قطع العرض",                              390, 45,  "2", 13),
            ("معالجة أى تسريب مياه من الروتارى جوينت DR",           360, 60,  "2", 13),
            ("التأكد من عمل الثرموريجيوليشن",                       470, 45,  "1", 8),
            ("معالجة أى تسريب مياه من الثرموريجيوليشن",             360, 60,  "1", 8)
        ]
    },
    # Image 2
    {
        "sub_name": "سير الماستر",
        "tasks": [
            ("ضبط حركة سير الماستر",                                300, 30,  "1", 8),
            ("تنظيف رولات سير الماستر",                             340, 60,  "1", 8),
            ("تغيير كتاين رولات سير الماستر",                       490, 60,  "2", 26)
        ]
    },
    {
        "sub_name": "بقية الخط",
        "tasks": [
            ("معالجة أى تسريب مياه من روتارى جوينت الكولنج درام",   360, 60,  "1", 13),
            ("تنظيف رولات وسير السنترة السفلى",                     340, 60,  "1", 8),
            ("ضبط حركة سير السنترة السفلى",                         300, 30,  "1", 8),
            ("تنظيف رولات وسير السنترة العلوى",                     340, 60,  "1", 8),
            ("ضبط حركة سير السنترة العلوى",                         300, 30,  "1", 8),
            ("تنظيف رولات وسير الاندر- لينر",                       340, 60,  "1", 8),
            ("ضبط حركة سير الاندر- لينر",                           300, 60,  "2", 13),
            ("تنظيف رولات وسير اللينر",                             340, 45,  "1", 8),
            ("ضبط حركة سير اللينر",                                 300, 60,  "2", 13),
            ("تنظيف رولات وسير Over - path اللينر",                 340, 60,  "1", 13),
            ("ضبط حركة سير Over - path اللينر",                     300, 60,  "2", 13),
            ("تنظيف رولات وسير الاندر- لينر Let-off1",              340, 60,  "1", 8),
            ("ضبط حركة سير الاندر- لينر Let-off1",                  300, 60,  "2", 13),
            ("تنظيف رولات وسير الاندر- لينر Let-off2",              340, 60,  "1", 8),
            ("ضبط حركة سير الاندر- لينر Let-off2",                  300, 60,  "2", 13),
            ("التأكد من عمل بساتم Let-off1 بكفاءة",                 420, 60,  "1", 13),
            ("التأكد من عمل بساتم Let-off2 بكفاءة",                 420, 60,  "1", 13),
            ("التأكد من عمل اسكرو حمل البوبينة Let-off1 بكفاءة",    500, 60,  "1", 13),
            ("التأكد من عمل اسكرو حمل البوبينة Let-off2 بكفاءة",    500, 60,  "1", 13),
            ("ضبط قطع سكينة الوايند-أب",                            380, 45,  "1", 13),
            ("تغيير سكينتى الوايند-أب (الروتارى و الكونتر)",        390, 120, "2", 26),
            ("تنظيف رولات وسير اللينر Wind-up1",                    340, 60,  "1", 8),
            ("ضبط حركة سير اللينر Wind-up1",                        300, 60,  "2", 13),
            ("تنظيف رولات وسير اللينر Wind-up2",                    340, 60,  "1", 8),
            ("ضبط حركة سير الاندر- لينر Wind-up2",                  300, 60,  "2", 13),
            ("التأكد من عمل بساتم Wind-up1 بكفاءة",                 420, 60,  "1", 13),
            ("التأكد من عمل بساتم Wind-up2 بكفاءة",                 420, 60,  "1", 13),
            ("التأكد من عمل اسكرو حمل البوبينة Let-off1 بكفاءة",    500, 60,  "1", 13),
            ("التأكد من عمل اسكرو حمل البوبينة Let-off2 بكفاءة",    500, 60,  "1", 13),
            ("تغيير اسكرو حمل البوبينة Let-off1",                   510, 120, "2", 26),
            ("تغيير اسكرو حمل البوبينة Let-off2",                   510, 120, "2", 26),
            ("تغيير اسكرو حمل البوبينة Wind-up1",                   510, 120, "2", 26),
            ("تغيير اسكرو حمل البوبينة Wind-up2",                   510, 120, "2", 26)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        if period == 26:
            start_week = 25
        elif period == 13:
            start_week = 1   
            step = 12  # Custom step for 13 weeks
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'Roller_Head_Mechanical_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\Roller_Head_Mechanical_PM_Plan.csv
Total Tasks Processed: 60


In [9]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "Triplex Mechanical"

submachines_data = [
    # Image 1
    {
        "sub_name": "سير التغذية",
        "tasks": [
            ("التأكد من عمل سير التغذية بكفاءة",                    300, 15,  "1", 13),
            ("تنظيف رولات سير التغذية",                             340, 60,  "1", 1),
            ("تنظيف سير التغذية",                                   350, 60,  "1", 8)
        ]
    },
    # Image 2
    {
        "sub_name": "الاكسترودر",
        "tasks": [
            ("معالجة اى تسريب زيت من وحدة الهيدروليك",              310, 60,  "1", 8),
            ("معالجة اى تسريب زيت من الجيربوكس",                    310, 60,  "2", 8),
            ("معالجة اى تسريب زيت من الفيدرول",                     310, 60,  "1", 8),
            ("تغيير بلى الفيدرول",                                  440, 120, "2", 26),
            ("معالجة أي تسريب مياه من روتارى جوينت الاكسترودر",     360, 60,  "2", 8),
            ("التأكد من عمل Driving Coupling بكفاءة",               350, 60,  "1", 26),
            ("التأكد من عمل الثرموريجيوليشن",                       470, 45,  "1", 1),
            ("معالجة أى تسريب مياه من الثرموريجيوليشن",             360, 60,  "2", 1)
        ]
    },
    # Image 3
    {
        "sub_name": "سير الماستر",
        "tasks": [
            ("ضبط حركة سير الماستر",                                300, 30,  "1", 8),
            ("تنظيف رولات سير الماستر",                             340, 60,  "1", 8),
            ("تغيير كتاين رولات سير الماستر",                       490, 60,  "2", 26)
        ]
    },
    {
        "sub_name": "سير الختم",
        "tasks": [
            ("ضبط حركة سير الختم",                                  300, 30,  "1", 8),
            ("تنظيف رولات سير الختم",                               340, 60,  "1", 8),
            ("تغيير كتاين رولات سير الختم",                         490, 60,  "2", 26)
        ]
    },
    {
        "sub_name": "الكالندر",
        "tasks": [
            ("معالجة اى تسريب زيت من وحدة هيدروليك الكالندر",       310, 60,  "1", 1),
            ("التأكد من ضغوط زيت الهيدروليك",                       410, 30,  "1", 1),
            ("معالجة اى تسريب زيت من وحدة تزييت الكالندر",          310, 60,  "1", 8),
            ("تنظيف فلاتر زيت وحدة تزييت الكالندر",                 400, 60,  "1", 13),
            ("التأكد من ضغوط زيت وحدة التزييت",                     410, 30,  "1", 8),
            # From Image 4
            ("التأكد من عمل البساتم بكفاءة",                        420, 60,  "1", 13),
            ("معالجة أى تسريب مياه من الروتارى جوينت OP",           360, 60,  "2", 13),
            ("ضبط الخلوص بين الدرافيل",                             430, 180, "2", 26),
            ("ضبط سكاكين قطع العرض",                                380, 45,  "1", 13),
            ("تغيير سكاكين قطع العرض",                              390, 45,  "2", 13),
            ("معالجة أى تسريب مياه من الروتارى جوينت DR",           360, 60,  "2", 13),
            ("التأكد من عمل الثرموريجيوليشن",                       470, 45,  "1", 1),
            ("معالجة أى تسريب مياه من الثرموريجيوليشن",             360, 60,  "1", 1)
        ]
    },
    # Image 4 continued
    {
        "sub_name": "بقية الخط",
        "tasks": [
            ("تنظيف رولات وسير و حوض التبريد العلوى",               340, 120, "4", 26),
            ("ضبط حركة سير حوض التبريد العلوى",                     300, 60,  "1", 13),
            ("تنظيف رولات وسير و حوض التبريد الاوسط",               340, 120, "4", 26),
            ("ضبط حركة سير حوض التبريد الاوسط",                     300, 60,  "1", 13),
            ("تنظيف رولات وسير و حوض التبريد السفلى",               340, 120, "4", 26),
            ("ضبط حركة سير حوض التبريد السفلى",                     300, 60,  "1", 13),
            ("تغيير كتاين رولات ما بعد حوض التبريد السفلى",         490, 60,  "2", 26),
            ("ضبط حركة سير السكينة",                                300, 60,  "2", 13),
            ("تغيير كتاين رولات ما بعد السكينة",                    490, 60,  "2", 26),
            ("ضبط قطع السكينة",                                     380, 60,  "1", 13),
            ("تغيير سكينتى (الروتارى و الكونتر)",                   390, 180, "2", 26),
            ("التأكد من خلوص اسكرو 150,200,180",                    580, 480, "6", 26),
            ("تغيير اسكرو 150,200,180",                             590, 480, "6", 52)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        if period == 26:
            start_week = 25
        elif period == 13:
            start_week = 1   
            step = 12  # Custom step for 13 weeks
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'Triplex_Mechanical_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\Triplex_Mechanical_PM_Plan.csv
Total Tasks Processed: 43


In [10]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "Triplex Lubrication"

submachines_data = [
    # Image 1
    {
        "sub_name": 'سير تغذية اكسترودر 8"',
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "خط انتاج الترد",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تنظيف",      600, 60, "1", 1),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "منطقة الاكسترودر",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "وحدة هيدروليك الاكسترودر",
        "tasks": [
            ("تنظيف",      600, 60, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    # Image 2
    {
        "sub_name": "تزييت الفيدرول",
        "tasks": [
            ("تنظيف",      600, 60, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سير الماستر",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "الميزان المترى",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سير الطباعة",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "احواض التبريد",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    # Image 3
    {
        "sub_name": "سير الرولات",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "سير الميزان",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "الكالندر",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "تزييت الكالندر",
        "tasks": [
            ("تنظيف",      600, 60, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "وحدة فرامل الكالندر",
        "tasks": [
            ("تنظيف",      600, 60, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    # Image 4
    {
        "sub_name": "C3 اكسترودر",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "C3 هيدروليك",
        "tasks": [
            ("تنظيف",      600, 60, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سير تغذية الكالندر",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سكينة قطع الترد",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "هيدروليك سكينة الترد",
        "tasks": [
            ("تنظيف",      600, 60, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        if period == 26:
            start_week = 25
        elif period == 13:
            start_week = 1   
            step = 12  # Custom step for 13 weeks
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'Triplex_Lubrication_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\Triplex_Lubrication_PM_Plan.csv
Total Tasks Processed: 41


In [11]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "Duplex Mechanical"

submachines_data = [
    # Image 1
    {
        "sub_name": "سير التغذية",
        "tasks": [
            ("التأكد من عمل سير التغذية بكفاءة",                    300, 15,  "1", 13),
            ("تنظيف رولات سير التغذية",                             340, 60,  "1", 8),
            ("تنظيف سير التغذية",                                   350, 60,  "1", 8)
        ]
    },
    {
        "sub_name": "الاكسترودر",
        "tasks": [
            ("معالجة اى تسريب زيت من وحدة الهيدروليك",              310, 60,  "1", 8),
            ("معالجة اى تسريب زيت من الجيربوكس",                    310, 60,  "2", 8),
            ("معالجة اى تسريب زيت من الفيدرول",                     310, 60,  "1", 8),
            ("تغيير بلى الفيدرول",                                  440, 120, "2", 26),
            ("معالجة أى تسريب مياه من روتارى جوينت الاكسترودر",     360, 60,  "2", 8),
            ("التأكد من عمل Driving Coupling بكفاءة",               350, 60,  "1", 26),
            ("التأكد من عمل الثرموريجيوليشن",                       470, 45,  "1", 8),
            ("معالجة أى تسريب مياه من الثرموريجيوليشن",             360, 60,  "2", 8)
        ]
    },
    {
        "sub_name": "سير الماستر",
        "tasks": [
            ("ضبط حركة سير الماستر",                                300, 30,  "1", 8),
            ("تنظيف رولات سير الماستر",                             340, 60,  "1", 8),
            ("تغيير كتاين رولات سير الماستر",                       490, 60,  "2", 26)
        ]
    },
    # Image 2
    {
        "sub_name": "سير الاسترب",
        "tasks": [
            ("ضبط حركة سير الاسترب",                                300, 30,  "1", 8),
            ("تنظيف رولات سير الاسترب",                             340, 60,  "1", 8),
            ("الكشف علي حالة فلتر الزيت",                           610, 30,  "1", 8),
            ("الكشف علي ضغط البريس لوحدة البريس",                   600, 30,  "1", 8),
            ("الكشف علي مسارات الهواء",                             620, 30,  "1", 8),
            ("تغيير كتاين رولات سير الاسترب",                       490, 60,  "2", 26)
        ]
    },
    {
        "sub_name": "بقية الخط",
        "tasks": [
            ("تنظيف رولات وسير و حوض التبريد العلوى",               340, 120, "4", 26),
            ("ضبط حركة سير حوض التبريد العلوى",                     300, 60,  "1", 13),
            ("تنظيف رولات وسير و حوض التبريد الاوسط",               340, 120, "4", 26),
            ("ضبط حركة سير حوض التبريد الاوسط",                     300, 60,  "1", 13),
            ("تنظيف رولات وسير و حوض التبريد السفلى",               340, 120, "4", 26),
            ("ضبط حركة سير حوض التبريد السفلى",                     300, 60,  "1", 13),
            ("تغيير كتاين رولات ما بعد حوض التبريد السفلى",         490, 60,  "2", 26),
            ("ضبط حركة سير السكينة",                                300, 60,  "2", 13),
            ("تغيير كتاين رولات ما بعد السكينة",                    490, 60,  "2", 26),
            ("ضبط قطع السكينة",                                     380, 60,  "1", 13),
            ("تغيير سكينتى (الروتارى و الكونتر)",                   390, 180, "2", 26),
            ("التأكد من خلوص اسكرو 120",                            580, 480, "6", 26),
            ("تغيير اسكرو 120",                                     590, 480, "6", 52),
            ("التأكد من خلوص اسكرو 150",                            580, 480, "6", 26),
            ("تغيير اسكرو 150",                                     590, 480, "6", 52)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        if period == 26:
            start_week = 25
        elif period == 13:
            start_week = 1   
            step = 12  # Custom step for 13 weeks
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'Duplex_Mechanical_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\Duplex_Mechanical_PM_Plan.csv
Total Tasks Processed: 35


In [12]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "Duplex Lubrication"

submachines_data = [
    # Image 1
    {
        "sub_name": 'سير تغذية اكسترودر "6',
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "خط انتاج السيدوول",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تنظيف",      600, 60, "1", 8),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "منطقة الاكسترودر",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 6),  # Note: Image shows Freq 6
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "وحدة هيدروليك الاكسترودر",
        "tasks": [
            ("تنظيف",      600, 60, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "وحدة تزييت الفيدرول",
        "tasks": [
            ("تنظيف",      600, 60, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سير الماستر",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    # Image 2
    {
        "sub_name": "سير الاسترب",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "السير الرافع",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "احواض التبريد",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "سير الرولات 1",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "سكينة قطع السيدوول",
        "tasks": [
            ("تشحيم",      630, 60, "1", 13), # Note: Code is 630 in image
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "وحدة هيدروليك السكينة",
        "tasks": [
            ("تنظيف",      600, 60, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سير الرولات 2",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        if period == 26:
            start_week = 25
        elif period == 13:
            start_week = 1   
            step = 12  # Custom step for 13 weeks
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'Duplex_Lubrication_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\Duplex_Lubrication_PM_Plan.csv
Total Tasks Processed: 28


In [13]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    # Ensure start_week doesn't exceed 52
    if start_week > 52:
        start_week = start_week % 52
        if start_week == 0: start_week = 52
        
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---

# Configuration for the 4 machines to stagger them
machines_config = [
    {"name": "C15.1", "start_offset": 1},
    {"name": "C15.2", "start_offset": 2},
    {"name": "C15.3", "start_offset": 3},
    {"name": "C15.4", "start_offset": 4}
]

# The tasks are identical for all C15 machines, so we define one template
# Structure: (Sub_Machine, Description, Code, Mins, Crew, Freq)
tasks_template = [
    ("Rewinding M/C", "تنظيف", 600, 60, "1", 8),
    ("Rewinding M/C", "تشحيم", 620, 60, "1", 13),
    ("Winder",        "تزييت", 630, 90, "1", 52),
    ("Winder",        "تشحيم", 620, 60, "1", 13)
]

data_rows = []

for mach in machines_config:
    machine_name = mach["name"]
    base_start = mach["start_offset"]
    
    # Counter for the tasks within this specific machine
    task_counter = 1
    
    for task in tasks_template:
        sub_name, desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        # Apply the machine's specific start offset
        current_start_week = base_start
        step = period 
        
        # Special User Rules applied on top of the offset:
        if period == 13:
            # Rule: Freq 13 has a step of 12 (repeats every 12 weeks)
            step = 12 
        
        # (Note: There were no Freq 26 tasks in these images, 
        # but if there were, we would likely add 24 to the base_start)

        row = {
            'Machine': machine_name,
            'Sub_Machine': sub_name, 
            '.': task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, current_start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C15_Mechanical_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {len(data_rows)}")

File saved successfully at: tabel\C15_Mechanical_PM_Plan.csv
Total Tasks Processed: 16


In [15]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---

# List of Machine Names
machines_list = ["C16.1", "C16.2", "C16.3", "C16.4"]

# Tasks Template (Identical for all C16 machines)
# Format: (Sub_Machine, Description, Code, Mins, Crew, Freq)
tasks_template = [
    ("Bead Wrapping M/C", "تنظيف", 600, 60, "1", 8),
    ("Bead Wrapping M/C", "تشحيم", 620, 60, "1", 13),
    ("Winder",            "تزييت", 630, 90, "1", 52),
    ("Winder",            "تشحيم", 620, 60, "1", 13)
]

data_rows = []

for machine_name in machines_list:
    # Counter for tasks within this specific machine
    task_counter = 1
    
    for task in tasks_template:
        sub_name, desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        # User request: "No staggering, all starts W1"
        start_week = 1
        step = period 
        
        # Apply special step logic for Freq 13 (repeats every 12 weeks to hit W1, W13, W25...)
        if period == 13:
            step = 12 
            
        row = {
            'Machine': machine_name,
            'Sub_Machine': sub_name, 
            '.': task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C16_Mechanical_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {len(data_rows)}")

File saved successfully at: tabel\C16_Mechanical_PM_Plan.csv
Total Tasks Processed: 16


In [16]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "C17 Mechanical"

submachines_data = [
    {
        "sub_name": "Autoclave M/C",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Hyd. Unit",
        "tasks": [
            ("تنظيف",      600, 60, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        # Apply special frequency rules
        if period == 13:
            step = 12  # Repeats every 12 weeks to hit W1, W13, W25...
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C17_Mechanical_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\C17_Mechanical_PM_Plan.csv
Total Tasks Processed: 4


In [17]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # Ensure start_week handles wrapping if offset pushes it beyond 52 (unlikely here but good practice)
    if start_week > 52:
        start_week = start_week % 52
        if start_week == 0: start_week = 52
        
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---

# Generate machine names C19.1 to C19.8
machines_list = [f"C19.{i}" for i in range(1, 9)]

# Task Template (Identical for all C19 machines based on images)
# Format: (Sub_Machine, Description, Code, Mins, Crew, Freq)
tasks_template = [
    ("Bead Filler M/Cs", "زيت الهواء", 610, 30, "1", 8),
    ("Bead Filler M/Cs", "تنظيف",      600, 60, "1", 8),
    ("Bead Filler M/Cs", "تشحيم",      620, 60, "1", 13),
    ("Winder",           "تزييت",      630, 90, "1", 52),
    ("Winder",           "تشحيم",      620, 60, "1", 13)
]

data_rows = []

for idx, machine_name in enumerate(machines_list):
    # --- Staggering Logic ---
    # idx 0 (C19.1) -> 0 % 4 = 0 -> start W1
    # idx 1 (C19.2) -> 1 % 4 = 1 -> start W2
    # ...
    # idx 4 (C19.5) -> 4 % 4 = 0 -> start W1 (Reset)
    base_start_week = (idx % 4) + 1
    
    task_counter = 1
    
    for task in tasks_template:
        sub_name, desc, code, mins, crew, period = task
        
        # Determine step size and start week
        step = period
        current_start = base_start_week
        
        # Special rule for Freq 13 (repeats every 12 weeks)
        if period == 13:
            step = 12
            
        row = {
            'Machine': machine_name,
            'Sub_Machine': sub_name, 
            '.': task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, current_start)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C19_Mechanical_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {len(data_rows)}")

File saved successfully at: tabel\C19_Mechanical_PM_Plan.csv
Total Tasks Processed: 40


In [23]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---

all_machines_data = [
    # ---------------------------------------------------------
    # MACHINE 1: C14.1 Mechanical
    # ---------------------------------------------------------
    {
        "machine_name": "C14.1 Mechanical",
        "submachines": [
            {
                "sub_name": "ماكينة تصنيع طوق اليد",
                "tasks": [
                    ("ضبط وحدة الليت-اوف",                                  380, 60,  "1", 4),
                    ("سنترة دخول السلك للاكسترودر",                         380, 60,  "2", 13),
                    ("معالجة اي تسريب زيت من الجيربوكس",                    310, 60,  "2", 26),
                    ("معالجة أي تسريب مياه من روتارى جوينت الاكسترودر",     360, 60,  "1", 13),
                    ("بكفاءة Driving Coupling التأكد من عمل",               380, 45,  "1", 13),
                    ("التأكد من عمل الثرموريجيوليشن",                       470, 60,  "2", 8),
                    ("معالجة أي تسريب مياه من الثرموريجيوليشن",             360, 60,  "1", 8),
                    ("الكشف على بلى طارات الفستون",                         440, 120, "1", 12),
                    ("ضبط اوضاع عمل الفستون",                               550, 60,  "2", 26),
                    ("معالجة اي تسريب زيت من وحدة الهيدروليك",              310, 60,  "1", 13),
                    ("تغيير كاتينة الفستون",                                490, 60,  "1", 26),
                    ("الكشف على طارات الفستون",                             440, 60,  "1", 12),
                    ("التأكد من عمل البساتم بكفاءة",                        420, 60,  "1", 13),
                    ("التأكد من اوضاع عمل الوايندر",                        550, 60,  "1", 13)
                ]
            }
        ]
    },
    # ---------------------------------------------------------
    # MACHINE 2: C14.1 Lubrication
    # ---------------------------------------------------------
    {
        "machine_name": "C14.1 Lubrication",
        "submachines": [
            {
                "sub_name": "خط تصنيع الطوق",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تنظيف",      600, 60, "1", 8),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "الاكسترودر",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "درامات التبريد",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "الفستونـة",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "تقديم السلك",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "وحدة الهيدروليك",
                "tasks": [
                    ("تنظيف",      600, 60, "1", 8),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "الوايندر",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            }
        ]
    }
]

data_rows = []

for machine in all_machines_data:
    mach_name = machine["machine_name"]
    global_task_counter = 1
    
    for sub in machine["submachines"]:
        sub_name = sub["sub_name"]
        for task in sub["tasks"]:
            desc, code, mins, crew, period = task
            
            # --- Logic for Start Week and Step ---
            start_week = 1
            step = period 
            
            if period == 26:
                start_week = 25
            elif period == 13:
                start_week = 1   
                step = 12  # Custom step for 13 weeks
            
            row = {
                'Machine': mach_name,
                'Sub_Machine': sub_name, 
                '.': global_task_counter,
                'Task_Description': desc,
                'Task_Code': code,
                'Estimated_mins': mins,
                'Crew': crew,
                'Period_freq': period
            }
            
            # Apply week flags
            week_flags = get_weeks(step, start_week)
            for w_idx, flag in enumerate(week_flags, 1):
                row[f'W{w_idx}'] = flag
                
            data_rows.append(row)
            global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C14_1_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {len(data_rows)}")

File saved successfully at: tabel\C14_1_PM_Plan.csv
Total Tasks Processed: 30


In [24]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---

all_machines_data = [
    # ---------------------------------------------------------
    # MACHINE 1: C14.2 Mechanical
    # ---------------------------------------------------------
    {
        "machine_name": "C14.2 Mechanical",
        "submachines": [
            {
                "sub_name": "ماكينة تصنيع طوق اليد",
                "tasks": [
                    ("ضبط وحدة الليت-اوف",                                  380, 60,  "1", 13),
                    ("سنترة دخول السلك للاكسترودر",                         380, 60,  "2", 13),
                    ("معالجة اي تسريب زيت من الجيربوكس",                    310, 60,  "2", 26),
                    ("معالجة أي تسريب مياه من روتارى جوينت الاكسترودر",     360, 60,  "1", 13),
                    ("بكفاءة Driving Coupling التأكد من عمل",380, 45,  "1", 13),
                    ("التأكد من عمل الثرموريجيوليشن",                       470, 60,  "2", 8),
                    ("معالجة أي تسريب مياه من الثرموريجيوليشن",             360, 60,  "1", 8),
                    ("تغيير بلى طارات الفستون",                             440, 120, "1", 26),
                    ("ضبط اوضاع عمل الفستون",                               550, 60,  "2", 26),
                    ("معالجة اي تسريب زيت من وحدة الهيدروليك",              310, 60,  "1", 13),
                    ("تغيير كاتينة الفستون",                                490, 60,  "1", 26),
                    ("تغيير طارات الفستون",                                 440, 60,  "1", 52),
                    ("التأكد من عمل البساتم بكفاءة",                        420, 60,  "1", 13),
                    ("التأكد من اوضاع عمل الوايندر",                        550, 60,  "1", 13)
                ]
            }
        ]
    },
    # ---------------------------------------------------------
    # MACHINE 2: C14.2 Lubrication
    # ---------------------------------------------------------
    {
        "machine_name": "C14.2 Lubrication",
        "submachines": [
            {
                "sub_name": "خط تصنيع الطوق",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تنظيف",      600, 60, "1", 8),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "الاكسترودر",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "درامات التبريد",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "الفستونـة",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "تقديم السلك",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "وحدة الهيدروليك",
                "tasks": [
                    ("تنظيف",      600, 60, "1", 8),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "الوايندر",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            }
        ]
    }
]

data_rows = []

for machine in all_machines_data:
    mach_name = machine["machine_name"]
    global_task_counter = 1
    
    for sub in machine["submachines"]:
        sub_name = sub["sub_name"]
        for task in sub["tasks"]:
            desc, code, mins, crew, period = task
            
            # --- Logic for Start Week and Step ---
            start_week = 1
            step = period 
            
            if period == 26:
                start_week = 25
            elif period == 13:
                start_week = 1   
                step = 12  # Custom step for 13 weeks
            
            row = {
                'Machine': mach_name,
                'Sub_Machine': sub_name, 
                '.': global_task_counter,
                'Task_Description': desc,
                'Task_Code': code,
                'Estimated_mins': mins,
                'Crew': crew,
                'Period_freq': period
            }
            
            # Apply week flags
            week_flags = get_weeks(step, start_week)
            for w_idx, flag in enumerate(week_flags, 1):
                row[f'W{w_idx}'] = flag
                
            data_rows.append(row)
            global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C14_2_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {len(data_rows)}")

File saved successfully at: tabel\C14_2_PM_Plan.csv
Total Tasks Processed: 30


In [25]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---

all_machines_data = [
    # ---------------------------------------------------------
    # MACHINE 1: C14.3 Mechanical
    # ---------------------------------------------------------
    {
        "machine_name": "C14.3 Mechanical",
        "submachines": [
            {
                "sub_name": "ماكينة تصنيع طوق اليد",
                "tasks": [
                    ("ضبط وحدة الليت-اوف",                                  380, 60,  "1", 4),
                    ("سنترة دخول السلك للاكسترودر",                         380, 60,  "2", 13),
                    ("معالجة اي تسريب زيت من الجيربوكس",                    310, 60,  "2", 26),
                    ("معالجة أي تسريب مياه من روتارى جوينت الاكسترودر",     360, 60,  "1", 13),
                    ("بكفاءة Driving Coupling التأكد من عمل",               380, 45,  "1", 13),
                    ("التأكد من عمل الثرموريجيوليشن",                       470, 60,  "2", 8),
                    ("معالجة أي تسريب مياه من الثرموريجيوليشن",             360, 60,  "1", 8),
                    ("الكشف على بلى طارات الفستون",                         440, 120, "1", 12),
                    ("ضبط اوضاع عمل الفستون",                               550, 60,  "2", 26),
                    ("معالجة اي تسريب زيت من وحدة الهيدروليك",              310, 60,  "1", 13),
                    ("تغيير كاتينة الفستون",                                490, 60,  "1", 26),
                    ("الكشف على طارات الفستون",                             440, 60,  "1", 12),
                    ("التأكد من عمل البساتم بكفاءة",                        420, 60,  "1", 13),
                    ("التأكد من اوضاع عمل الوايندر",                        550, 60,  "1", 13)
                ]
            }
        ]
    },
    # ---------------------------------------------------------
    # MACHINE 2: C14.3 Lubrication
    # ---------------------------------------------------------
    {
        "machine_name": "C14.3 Lubrication",
        "submachines": [
            {
                "sub_name": "خط تصنيع الطوق",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تنظيف",      600, 60, "1", 8),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "الاكسترودر",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "درامات التبريد",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "الفستونـة",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "تقديم السلك",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "وحدة الهيدروليك",
                "tasks": [
                    ("تنظيف",      600, 60, "1", 8),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "الوايندر",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            }
        ]
    }
]

data_rows = []

for machine in all_machines_data:
    mach_name = machine["machine_name"]
    global_task_counter = 1
    
    for sub in machine["submachines"]:
        sub_name = sub["sub_name"]
        for task in sub["tasks"]:
            desc, code, mins, crew, period = task
            
            # --- Logic for Start Week and Step ---
            start_week = 1
            step = period 
            
            if period == 26:
                start_week = 25
            elif period == 13:
                start_week = 1   
                step = 12  # Custom step for 13 weeks
            elif period == 12:
                step = 12 # Standard 12 week interval
            
            row = {
                'Machine': mach_name,
                'Sub_Machine': sub_name, 
                '.': global_task_counter,
                'Task_Description': desc,
                'Task_Code': code,
                'Estimated_mins': mins,
                'Crew': crew,
                'Period_freq': period
            }
            
            # Apply week flags
            week_flags = get_weeks(step, start_week)
            for w_idx, flag in enumerate(week_flags, 1):
                row[f'W{w_idx}'] = flag
                
            data_rows.append(row)
            global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C14_3_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {len(data_rows)}")

File saved successfully at: tabel\C14_3_PM_Plan.csv
Total Tasks Processed: 30


In [27]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---

all_machines_data = [
    # ---------------------------------------------------------
    # MACHINE 1: C14.4 Mechanical
    # ---------------------------------------------------------
    {
        "machine_name": "C14.4 Mechanical",
        "submachines": [
            {
                "sub_name": "ماكينة تصنيع طوق اليد",
                "tasks": [
                    ("ضبط وحدة الليت-اوف",                                  380, 60,  "1", 4),
                    ("سنترة دخول السلك للاكسترودر",                         380, 60,  "2", 13),
                    ("معالجة اي تسريب زيت من الجيربوكس",                    310, 60,  "2", 26),
                    ("معالجة أي تسريب مياه من روتارى جوينت الاكسترودر",     360, 60,  "1", 13),
                    ("بكفاءة Driving Coupling التأكد من عمل",               380, 45,  "1", 13),
                    ("التأكد من عمل الثرموريجيوليشن",                       470, 60,  "2", 8),
                    ("معالجة أي تسريب مياه من الثرموريجيوليشن",             360, 60,  "1", 8),
                    ("الكشف على بلى طارات الفستون",                         440, 120, "1", 12),
                    ("ضبط اوضاع عمل الفستون",                               550, 60,  "2", 26),
                    ("معالجة اي تسريب زيت من وحدة الهيدروليك",              310, 60,  "1", 13),
                    ("تغيير كاتينة الفستون",                                490, 60,  "1", 26),
                    ("الكشف على طارات الفستون",                             440, 60,  "1", 12),
                    ("التأكد من عمل البساتم بكفاءة",                        420, 60,  "1", 13),
                    ("التأكد من اوضاع عمل الوايندر",                        550, 60,  "1", 13)
                ]
            }
        ]
    },
    # ---------------------------------------------------------
    # MACHINE 2: C14.4 Lubrication
    # ---------------------------------------------------------
    {
        "machine_name": "C14.4 Lubrication",
        "submachines": [
            {
                "sub_name": "خط تصنيع الطوق",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تنظيف",      600, 60, "1", 8),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "الاكسترودر",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "درامات التبريد",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "الفستونـة",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "تقديم السلك",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "وحدة الهيدروليك",
                "tasks": [
                    ("تنظيف",      600, 60, "1", 8),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "الوايندر",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            }
        ]
    }
]

data_rows = []

for machine in all_machines_data:
    mach_name = machine["machine_name"]
    global_task_counter = 1
    
    for sub in machine["submachines"]:
        sub_name = sub["sub_name"]
        for task in sub["tasks"]:
            desc, code, mins, crew, period = task
            
            # --- Logic for Start Week and Step ---
            start_week = 1
            step = period 
            
            if period == 26:
                start_week = 25
            elif period == 13:
                start_week = 1   
                step = 12  # Custom step for 13 weeks
            elif period == 12:
                step = 12 # Standard 12 week interval
            
            row = {
                'Machine': mach_name,
                'Sub_Machine': sub_name, 
                '.': global_task_counter,
                'Task_Description': desc,
                'Task_Code': code,
                'Estimated_mins': mins,
                'Crew': crew,
                'Period_freq': period
            }
            
            # Apply week flags
            week_flags = get_weeks(step, start_week)
            for w_idx, flag in enumerate(week_flags, 1):
                row[f'W{w_idx}'] = flag
                
            data_rows.append(row)
            global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C14_4_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {len(data_rows)}")

File saved successfully at: tabel\C14_4_PM_Plan.csv
Total Tasks Processed: 30


In [28]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---

# Machines listed in the image
machines_list = ["G1.1", "G1.2"]

# Tasks Template (Identical for G1.1 and G1.2)
tasks_template = [
    # Sub_Machine: Bladder M/C (ماكينة البلا در)
    ("ماكينة البلا در", "زيت الهواء", 610, 30, "1", 8),
    ("ماكينة البلا در", "تشحيم",      620, 60, "1", 13),
    
    # Sub_Machine: Winder & Pressure Arms (الوايندر و أذرع الضغط)
    ("الوايندر و أذرع الضغط", "تزييت", 630, 90, "1", 52),
    ("الوايندر و أذرع الضغط", "تشحيم", 620, 60, "1", 13)
]

data_rows = []

for machine_name in machines_list:
    # Task counter specific to each machine
    task_counter = 1
    
    for task in tasks_template:
        sub_name, desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        # Special rule for Freq 13 (repeats every 12 weeks)
        if period == 13:
            step = 12
            
        row = {
            'Machine': machine_name,
            'Sub_Machine': sub_name, 
            '.': task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'G1_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {len(data_rows)}")

File saved successfully at: tabel\G1_PM_Plan.csv
Total Tasks Processed: 8


In [29]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---

all_machines_data = [
    # ---------------------------------------------------------
    # MACHINE 1: C13
    # ---------------------------------------------------------
    {
        "machine_name": "C13",
        "submachines": [
            {
                "sub_name": "ماكينة سبليتر المطاط C13",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            }
        ]
    },
    # ---------------------------------------------------------
    # MACHINE 2: C12
    # ---------------------------------------------------------
    {
        "machine_name": "C12",
        "submachines": [
            {
                "sub_name": "ماكينة السبليتر",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تنظيف",      600, 60, "1", 8),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "وحدة الليت-اوف",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "وحدة القطع",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "وحدة الهيدروليك",
                "tasks": [
                    ("تنظيف",      600, 60, "1", 8),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "الوايندر",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            }
        ]
    }
]

data_rows = []

for machine in all_machines_data:
    mach_name = machine["machine_name"]
    global_task_counter = 1
    
    for sub in machine["submachines"]:
        sub_name = sub["sub_name"]
        for task in sub["tasks"]:
            desc, code, mins, crew, period = task
            
            # --- Logic for Start Week and Step ---
            start_week = 1
            step = period 
            
            if period == 26:
                start_week = 25
            elif period == 13:
                start_week = 1   
                step = 12  # Custom step for 13 weeks
            
            row = {
                'Machine': mach_name,
                'Sub_Machine': sub_name, 
                '.': global_task_counter,
                'Task_Description': desc,
                'Task_Code': code,
                'Estimated_mins': mins,
                'Crew': crew,
                'Period_freq': period
            }
            
            # Apply week flags
            week_flags = get_weeks(step, start_week)
            for w_idx, flag in enumerate(week_flags, 1):
                row[f'W{w_idx}'] = flag
                
            data_rows.append(row)
            global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C12_C13_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {len(data_rows)}")

File saved successfully at: tabel\C12_C13_PM_Plan.csv
Total Tasks Processed: 15


In [30]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "C1 Mechanical"

submachines_data = [
    # Image 1
    {
        "sub_name": "سير التغذية",
        "tasks": [
            ("التأكد من عمل سير التغذية بكفاءة",                    300, 15,  "1", 13),
            ("تنظيف رولات سير التغذية",                             340, 60,  "1", 1),
            ("تنظيف سير التغذية",                                   350, 60,  "1", 8)
        ]
    },
    {
        "sub_name": "الاكسترودر",
        "tasks": [
            ("معالجة اى تسريب زيت من وحدة الهيدروليك",              310, 60,  "1", 8),
            ("معالجة اى تسريب زيت من الجيربوكس",                    310, 60,  "2", 8),
            ("معالجة اى تسريب زيت من الفيدرول",                     310, 60,  "1", 8),
            ("تغيير بلى الفيدرول",                                  440, 120, "2", 26),
            ("معالجة أي تسريب مياه من روتارى جوينت الاكسترودر",     360, 60,  "2", 8),
            ("التأكد من عمل Driving Coupling بكفاءة",               350, 60,  "1", 26),
            ("التأكد من عمل الثرموريجيوليشن",                       470, 45,  "1", 1),
            ("معالجة أي تسريب مياه من الثرموريجيوليشن",             360, 60,  "2", 1)
        ]
    },
    # Image 2
    {
        "sub_name": "سير الماستر",
        "tasks": [
            ("ضبط حركة سير الماستر",                                300, 30,  "1", 8),
            ("تنظيف رولات سير الماستر",                             340, 60,  "1", 8),
            ("تغيير كتاين رولات سير الماستر",                       490, 60,  "2", 26)
        ]
    },
    {
        "sub_name": "سير الختم",
        "tasks": [
            ("ضبط حركة سير الختم",                                  300, 30,  "1", 8),
            ("تنظيف رولات سير الختم",                               340, 60,  "1", 8),
            ("تغيير كتاين رولات سير الختم",                         490, 60,  "2", 26)
        ]
    },
    {
        "sub_name": "الكالندر",
        "tasks": [
            ("معالجة اى تسريب زيت من وحدة هيدروليك الكالندر",       310, 60,  "1", 1),
            ("التأكد من ضغوط زيت الهيدروليك",                       410, 30,  "1", 1),
            ("معالجة اى تسريب زيت من وحدة تزييت الكالندر",          310, 60,  "1", 8),
            ("تنظيف فلاتر زيت وحدة تزييت الكالندر",                 400, 60,  "1", 13),
            ("التأكد من ضغوط زيت وحدة التزييت",                     410, 30,  "1", 8),
            # From Image 3
            ("التأكد من عمل البساتم بكفاءة",                        420, 60,  "1", 13),
            ("معالجة أي تسريب مياه من الروتارى جوينت OP",           360, 60,  "2", 13),
            ("ضبط الخلوص بين الدرافيل",                             430, 180, "2", 26),
            ("ضبط سكاكين قطع العرض",                                380, 45,  "1", 13),
            ("تغيير سكاكين قطع العرض",                              390, 45,  "2", 13),
            ("معالجة أي تسريب مياه من الروتارى جوينت DR",           360, 60,  "2", 13)
        ]
    },
    # Image 3 continued
    {
        "sub_name": "بقية الخط",
        "tasks": [
            ("التأكد من عمل الثرموريجيوليشن",                       470, 45,  "1", 1),
            ("معالجة أي تسريب مياه من الثرموريجيوليشن",             360, 60,  "1", 1),
            ("تنظيف رولات وسير و حوض التبريد العلوى",               340, 120, "4", 26),
            ("ضبط حركة سير حوض التبريد العلوى",                     300, 60,  "1", 13),
            ("تنظيف رولات وسير و حوض التبريد الاوسط",               340, 120, "4", 26),
            ("ضبط حركة سير حوض التبريد الاوسط",                     300, 60,  "1", 13),
            ("تنظيف رولات وسير و حوض التبريد السفلى",               340, 120, "4", 26),
            ("ضبط حركة سير حوض التبريد السفلى",                     300, 60,  "1", 13),
            ("تغيير كتاين رولات ما بعد حوض التبريد السفلى",         490, 60,  "2", 26),
            ("ضبط حركة سير السكينة",                                300, 60,  "2", 13),
            ("تغيير كتاين رولات ما بعد السكينة",                    490, 60,  "2", 26),
            ("ضبط قطع السكينة",                                     380, 60,  "1", 13),
            ("تغيير سكينتى (الروتارى و الكونتر)",                   390, 180, "2", 26),
            ("التأكد من خلوص اسكرو 150,200,120",                    580, 480, "6", 26),
            ("تغيير اسكرو 150,200,120",                             590, 480, "6", 52)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        if period == 26:
            start_week = 25
        elif period == 13:
            start_week = 1   
            step = 12  # Custom step for 13 weeks
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C1_Mechanical_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\C1_Mechanical_PM_Plan.csv
Total Tasks Processed: 43


In [31]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "C1 Lubrication"

submachines_data = [
    # Image 1
    {
        "sub_name": 'سير تغذية اكسترودر"8',
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "خط انتاج الترد",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تنظيف",      600, 60, "1", 1),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "منطقة الاكسترودر",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "وحدة هيدروليك الاكسترودر",
        "tasks": [
            ("تنظيف",      600, 60, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "تزييت الفيدرول",
        "tasks": [
            ("تنظيف",      600, 60, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سير الماستر",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    # Image 2
    {
        "sub_name": "الميزان المترى",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سير الطباعة",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "احواض التبريد",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "سير الرولات",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "سير الميزان",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "الكالندر",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    # Image 3
    {
        "sub_name": "تزييت الكالندر",
        "tasks": [
            ("تنظيف",      600, 60, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "وحدة فرامل الكالندر",
        "tasks": [
            ("تنظيف",      600, 60, "1", 8),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "C3 اكسترودر",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "C3 هيدروليك",
        "tasks": [
            ("تنظيف",      600, 60, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سير تغذية الكالندر",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "سكينة قطع الترد",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "هيدروليك سكينة الترد",
        "tasks": [
            ("تنظيف",      600, 60, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        if period == 26:
            start_week = 25
        elif period == 13:
            start_week = 1   
            step = 12  # Custom step for 13 weeks
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C1_Lubrication_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\C1_Lubrication_PM_Plan.csv
Total Tasks Processed: 41


In [35]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---

all_machines_data = [
    # ---------------------------------------------------------
    # MACHINE 1: C4 Mechanical
    # ---------------------------------------------------------
    {
        "machine_name": "C4 Mechanical",
        "submachines": [
            {
                "sub_name": "سير التغذية",
                "tasks": [
                    ("التأكد من عمل سير التغذية بكفاءة",                    300, 15,  "1", 13),
                    ("تنظيف رولات سير التغذية",                             340, 60,  "1", 1),
                    ("تنظيف سير التغذية",                                   350, 60,  "1", 8)
                ]
            },
            {
                "sub_name": "الاكسترودر",
                "tasks": [
                    ("معالجة اى تسريب زيت من وحدة الهيدروليك",              310, 60,  "1", 8),
                    ("معالجة اى تسريب زيت من الجيربوكس",                    310, 60,  "2", 8),
                    ("معالجة اى تسريب زيت من الفيدرول",                     310, 60,  "1", 8),
                    ("تغيير بلى الفيدرول",                                  440, 120, "2", 26),
                    ("معالجة أى تسريب مياه من روتارى جوينت الاكسترودر",     360, 60,  "2", 8),
                    ("بكفاءة Driving Coupling التأكد من عمل ",               350, 60,  "1", 26),
                    ("التأكد من عمل الثرموريجيوليشن",                       470, 45,  "1", 1),
                    ("معالجة أى تسريب مياه من الثرموريجيوليشن",             360, 60,  "2", 1)
                ]
            },
            {
                "sub_name": "سير الماستر",
                "tasks": [
                    ("ضبط حركة سير الماستر",                                300, 30,  "1", 8),
                    ("تنظيف رولات سير الماستر",                             340, 60,  "1", 8),
                    ("تغيير كتاين رولات سير الماستر",                       490, 60,  "2", 26)
                ]
            },
            {
                "sub_name": "سير الاسترب",
                "tasks": [
                    ("ضبط حركة سير الاسترب",                                300, 30,  "1", 8),
                    ("تنظيف رولات سير الاسترب",                             340, 60,  "1", 8),
                    ("تغيير كتاين رولات سير الاسترب",                       490, 60,  "2", 26)
                ]
            },
            {
                "sub_name": "بقية الخط",
                "tasks": [
                    ("تنظيف رولات وسير و حوض التبريد العلوى",               340, 120, "4", 26),
                    ("ضبط حركة سير حوض التبريد العلوى",                     300, 60,  "1", 13),
                    ("تنظيف رولات وسير و حوض التبريد الاوسط",               340, 120, "4", 26),
                    ("ضبط حركة سير حوض التبريد الاوسط",                     300, 60,  "1", 13),
                    ("تنظيف رولات وسير و حوض التبريد السفلى",               340, 120, "4", 26),
                    ("ضبط حركة سير حوض التبريد السفلى",                     300, 60,  "1", 13),
                    ("تغيير كتاين رولات ما بعد حوض التبريد السفلى",         490, 60,  "2", 26),
                    ("ضبط حركة سير السكينة",                                300, 60,  "2", 13),
                    ("تغيير كتاين رولات ما بعد السكينة",                    490, 60,  "2", 26),
                    ("ضبط قطع السكينة",                                     380, 60,  "1", 13),
                    ("تغيير سكينتى (الروتارى و الكونتر)",                   390, 180, "2", 26),
                    ("التأكد من خلوص اسكرو 120",                            580, 480, "6", 26),
                    ("تغيير اسكرو 120",                                     590, 480, "6", 52),
                    ("التأكد من خلوص اسكرو 150",                            580, 480, "6", 26),
                    ("تغيير اسكرو 150",                                     590, 480, "6", 52)
                ]
            }
        ]
    },
    # ---------------------------------------------------------
    # MACHINE 2: C4 Lubrication
    # ---------------------------------------------------------
    {
        "machine_name": "C4 Lubrication",
        "submachines": [
            {
                "sub_name": 'سير تغذية اكسترودر 6"',
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "خط انتاج السيدوول",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تنظيف",      600, 60, "1", 1),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "منطقة الاكسترودر",
                "tasks": [
                    ("زيت الهواء", 610, 30, "1", 8),
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "وحدة هيدروليك الاكسترودر",
                "tasks": [
                    ("تنظيف",      600, 60, "1", 1),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "وحدة تزييت الفيدرول",
                "tasks": [
                    ("تنظيف",      600, 60, "1", 1),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "سير الماستر",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "سير الاسترب",
                "tasks": [
                    ("تشحيم",      620, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "السير الرافع",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "احواض التبريد",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "سير الرولات 1",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            },
            {
                "sub_name": "سكينة قطع السيدوول",
                "tasks": [
                    ("تشحيم",      630, 60, "1", 13),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "وحدة هيدروليك السكينة",
                "tasks": [
                    ("تنظيف",      600, 60, "1", 1),
                    ("تزييت",      630, 90, "1", 52)
                ]
            },
            {
                "sub_name": "سير الرولات 2",
                "tasks": [
                    ("تزييت",      630, 90, "1", 52),
                    ("تشحيم",      620, 60, "1", 13)
                ]
            }
        ]
    }
]

data_rows = []

for machine in all_machines_data:
    mach_name = machine["machine_name"]
    global_task_counter = 1
    
    for sub in machine["submachines"]:
        sub_name = sub["sub_name"]
        for task in sub["tasks"]:
            desc, code, mins, crew, period = task
            
            # --- Logic for Start Week and Step ---
            start_week = 1
            step = period 
            
            if period == 26:
                start_week = 25
            elif period == 13:
                start_week = 1   
                step = 12  # Custom step for 13 weeks
            
            row = {
                'Machine': mach_name,
                'Sub_Machine': sub_name, 
                '.': global_task_counter,
                'Task_Description': desc,
                'Task_Code': code,
                'Estimated_mins': mins,
                'Crew': crew,
                'Period_freq': period
            }
            
            # Apply week flags
            week_flags = get_weeks(step, start_week)
            for w_idx, flag in enumerate(week_flags, 1):
                row[f'W{w_idx}'] = flag
                
            data_rows.append(row)
            global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C4_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {len(data_rows)}")

File saved successfully at: tabel\C4_PM_Plan.csv
Total Tasks Processed: 60


In [36]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    # range(start, stop, step)
    indices = range(start_week - 1, 52, step)
    
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "C5 Mechanical"

submachines_data = [
    # Image 1
    {
        "sub_name": "سير التغذية",
        "tasks": [
            ("التأكد من عمل سير التغذية بكفاءة",                    300, 15,  "1", 13),
            ("تنظيف رولات سير التغذية",                             340, 60,  "1", 8),
            ("تنظيف سير التغذية",                                   350, 60,  "1", 8)
        ]
    },
    {
        "sub_name": "الاكسترودر",
        "tasks": [
            ("معالجة اى تسريب زيت من وحدة الهيدروليك",              310, 60,  "1", 8),
            ("معالجة اى تسريب زيت من الجيربوكس",                    310, 60,  "2", 8),
            ("معالجة اى تسريب زيت من الفيدرول",                     310, 60,  "1", 8),
            ("تغيير بلى الفيدرول",                                  440, 120, "2", 13),
            ("معالجة أى تسريب مياه من روتارى جوينت الاكسترودر",     360, 60,  "2", 8),
            ("بكفاءة Driving التأكد من عمل سيور ",                   350, 60,  "1", 13),
            ("التأكد من عمل الثرموريجيوليشن",                       470, 45,  "1", 8),
            ("معالجة أى تسريب مياه من الثرموريجيوليشن",             360, 60,  "2", 8)
        ]
    },
    {
        "sub_name": "الكالندر",
        "tasks": [
            ("معالجة اى تسريب زيت من وحدة هيدروليك الكالندر",       310, 60,  "1", 8),
            ("التأكد من ضغوط زيت الهيدروليك",                       410, 30,  "1", 8),
            ("معالجة اى تسريب زيت من وحدة تزييت الكالندر",          310, 60,  "1", 8),
            ("تنظيف فلاتر زيت وحدة تزييت الكالندر",                 400, 60,  "1", 13),
            ("التأكد من ضغوط زيت وحدة التزييت",                     410, 30,  "1", 4),
            ("التأكد من عمل البساتم بكفاءة",                        420, 60,  "1", 13),
            ("OP معالجة أى تسريب مياه من الروتارى جوينت ",           360, 60,  "2", 13),
            ("ضبط الخلوص بين الدرافيل",                             430, 180, "2", 26),
            ("ضبط سكاكين قطع العرض",                                380, 45,  "1", 13),
            ("تغيير سكاكين قطع العرض",                              390, 45,  "2", 13),
            ("DR معالجة أى تسريب مياه من الروتارى جوينت ",           360, 60,  "2", 13),
            ("التأكد من عمل الثرموريجيوليشن",                       470, 45,  "1", 8),
            ("معالجة أى تسريب مياه من الثرموريجيوليشن",             360, 60,  "1", 8)
        ]
    },
    # Image 2
    {
        "sub_name": "سير الماستر",
        "tasks": [
            ("ضبط حركة سير الماستر",                                300, 30,  "1", 8),
            ("تنظيف رولات سير الماستر",                             340, 60,  "1", 8),
            ("تغيير كتاين رولات سير الماستر",                       490, 60,  "2", 26)
        ]
    },
    {
        "sub_name": "بقية الخط",
        "tasks": [
            ("معالجة أى تسريب مياه من روتارى جوينت الكولنج درام",   360, 60,  "1", 13),
            ("تنظيف رولات وسير السنترة السفلى", 340, 60,  "1", 1),
            ("ضبط حركة سير السنترة السفلى", 300, 30,  "1", 1),
            ("تنظيف رولات وسير السنترة العلوى",                     340, 60,  "1", 1),
            ("ضبط حركة سير السنترة العلوى",                         300, 30,  "1", 1),
            ("تنظيف رولات وسير الاندر- لينر",                       340, 60,  "1", 8),
            ("ضبط حركة سير الاندر- لينر",                           300, 60,  "2", 13),
            ("تنظيف رولات وسير اللينر",                             340, 45,  "1", 8),
            ("ضبط حركة سير اللينر",                                 300, 60,  "2", 13),
            ("Over - path تنظيف رولات وسير اللينر ",                 340, 60,  "1", 13),
            ("Over - path ضبط حركة سير اللينر ",                     300, 60,  "2", 13),
            ("Let-off1 تنظيف رولات وسير الاندر- لينر ",              340, 60,  "1", 8),
            ("Let-off1 ضبط حركة سير الاندر- لينر ",                  300, 60,  "2", 13),
            ("Let-off2 تنظيف رولات وسير الاندر- لينر ",              340, 60,  "1", 8),
            ("Let-off2 ضبط حركة سير الاندر- لينر ",                  300, 60,  "2", 13),
            ("التأكد من عمل بساتم Let-off1 بكفاءة",                 420, 60,  "1", 13),
            ("التأكد من عمل بساتم Let-off2 بكفاءة",                 420, 60,  "1", 13),
            ("بكفاءة Let-off1 التأكد من عمل اسكرو حمل البوبينة",    500, 60,  "1", 13),
            ("بكفاءة Let-off2 التأكد من عمل اسكرو حمل البوبينة ",    500, 60,  "1", 13),
            ("ضبط قطع سكينة الوايند-أب",                            380, 45,  "1", 13),
            ("تغيير سكينتى الوايند-أب (الروتارى و الكونتر)",        390, 120, "2", 26),
            ("Wind-up1 تنظيف رولات وسير اللينر ",                    340, 60,  "1", 8),
            ("Wind-up1 ضبط حركة سير اللينر ",                        300, 60,  "2", 13),
            ("Wind-up2 تنظيف رولات وسير اللينر ",                    340, 60,  "1", 8),
            ("Wind-up2 ضبط حركة سير اللينر ",                        300, 60,  "2", 13),
            ("بكفاءة Wind-up1 التأكد من عمل بساتم ",                 420, 60,  "1", 13),
            ("بكفاءة Wind-up2 التأكد من عمل بساتم ",                 420, 60,  "1", 13),
            ("بكفاءة Let-off1 التأكد من عمل اسكرو حمل البوبينة ",    500, 60,  "1", 13),
            ("بكفاءة Let-off2 التأكد من عمل اسكرو حمل البوبينة ",    500, 60,  "1", 13),
            ("Let-off1 تغيير اسكرو حمل البوبينة ",                   510, 120, "2", 26),
            ("Let-off2 تغيير اسكرو حمل البوبينة ",                   510, 120, "2", 26),
            ("Wind-up1 تغيير اسكرو حمل البوبينة ",                   510, 120, "2", 26),
            ("Wind-up2 تغيير اسكرو حمل البوبينة ",                   510, 120, "2", 26)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        
        # --- Logic for Start Week and Step ---
        start_week = 1
        step = period 
        
        if period == 26:
            start_week = 25
        elif period == 13:
            start_week = 1   
            step = 12  # Custom step for 13 weeks
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        # Apply week flags
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C5_Mechanical_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")
print(f"Total Tasks Processed: {global_task_counter - 1}")

File saved successfully at: tabel\C5_Mechanical_PM_Plan.csv
Total Tasks Processed: 60


In [37]:
import pandas as pd
import os

# Define the columns
columns = [
    'Machine', 'Sub_Machine', '.', 'Task_Description', 'Task_Code',
    'Estimated_mins', 'Crew', 'Period_freq',
] + [f'W{i}' for i in range(1, 53)]

# Helper function to generate week flags
def get_weeks(step, start_week):
    weeks = [''] * 52
    indices = range(start_week - 1, 52, step)
    for i in indices:
        weeks[i] = 'x'
    return weeks

# --- DATA ENTRY SECTION ---
main_machine = "C5 Lubrication"

submachines_data = [
    {
        "sub_name": '6" Ext. Feed. Conveyors',
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Liner Line",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تنظيف",      600, 60, "1", 13),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Extruder Zone",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Ext. Head Hyd. Unit",
        "tasks": [
            ("تنظيف",      600, 60, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Feed Roll Lub. Unit",
        "tasks": [
            ("تنظيف",      600, 60, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "2 - Roll Calender",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Calender Lub. Unit",
        "tasks": [
            ("تنظيف",      600, 30, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Cal. Hyd. Unit",
        "tasks": [
            ("تنظيف",      600, 60, "1", 1),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Master Conveyor",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Cooling Drums",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Under Belt Conveyor",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Coupling Conveyor",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Centering Conveyors",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Over - Head Conveyor",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Let-Off Cars",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Let-Off Conveyors",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Let-Off Festoon",
        "tasks": [
            ("تزييت",      630, 90, "1", 52),
            ("تشحيم",      620, 60, "1", 13)
        ]
    },
    {
        "sub_name": "Cutting Festoon",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Cutting M/C",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Wind-Up Conveyors",
        "tasks": [
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    },
    {
        "sub_name": "Wind-Up Cars",
        "tasks": [
            ("زيت الهواء", 610, 30, "1", 8),
            ("تشحيم",      620, 60, "1", 13),
            ("تزييت",      630, 90, "1", 52)
        ]
    }
]

data_rows = []
global_task_counter = 1

for sub in submachines_data:
    sub_name = sub["sub_name"]
    for task in sub["tasks"]:
        desc, code, mins, crew, period = task
        start_week = 1
        step = period 
        if period == 26:
            start_week = 25
        elif period == 13:
            start_week = 1   
            step = 12
            
        row = {
            'Machine': main_machine,
            'Sub_Machine': sub_name, 
            '.': global_task_counter,
            'Task_Description': desc,
            'Task_Code': code,
            'Estimated_mins': mins,
            'Crew': crew,
            'Period_freq': period
        }
        
        week_flags = get_weeks(step, start_week)
        for w_idx, flag in enumerate(week_flags, 1):
            row[f'W{w_idx}'] = flag
            
        data_rows.append(row)
        global_task_counter += 1

# Create DataFrame
df = pd.DataFrame(data_rows, columns=columns)

# --- SAVE TO FOLDER ---
output_folder = 'tabel'
file_name = 'C5_Lubrication_PM_Plan.csv'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

full_path = os.path.join(output_folder, file_name)
df.to_csv(full_path, index=False, encoding='utf-8-sig')

print(f"File saved successfully at: {full_path}")

File saved successfully at: tabel\C5_Lubrication_PM_Plan.csv


In [38]:
import pandas as pd
import os

# Configuration
folder_path = 'tabel'
output_filename = 'SF_Mech_PM_Plan.csv'
output_path = os.path.join(folder_path, output_filename)

# List to hold dataframes
all_dfs = []

# Check if folder exists
if os.path.exists(folder_path):
    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        # Process only CSV files, and skip the output file itself to avoid duplication
        if filename.endswith(".csv") and filename != output_filename:
            file_path = os.path.join(folder_path, filename)
            
            try:
                # Read the CSV
                df = pd.read_csv(file_path)
                all_dfs.append(df)
                print(f"Loaded: {filename} ({len(df)} rows)")
            except Exception as e:
                print(f"Error reading {filename}: {e}")

    # Combine all dataframes if any were found
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        
        # Save to the master CSV
        combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print("-" * 30)
        print(f"Success! Combined {len(all_dfs)} files.")
        print(f"Total rows: {len(combined_df)}")
        print(f"Saved to: {output_path}")
    else:
        print("No CSV files found to combine.")
else:
    print(f"The folder '{folder_path}' does not exist.")

Loaded: B1_Lubrication_Plan.csv (50 rows)
Loaded: B1_Mills_PM_Plan_Complete.csv (58 rows)
Loaded: B3_0_BELT_PM_Plan.csv (44 rows)
Loaded: C12_C13_PM_Plan.csv (15 rows)
Loaded: C14_1_PM_Plan.csv (30 rows)
Loaded: C14_2_PM_Plan.csv (30 rows)
Loaded: C14_3_PM_Plan.csv (30 rows)
Loaded: C14_4_PM_Plan.csv (30 rows)
Loaded: C15_Mechanical_PM_Plan.csv (16 rows)
Loaded: C16_Mechanical_PM_Plan.csv (16 rows)
Loaded: C17_Mechanical_PM_Plan.csv (4 rows)
Loaded: C19_Mechanical_PM_Plan.csv (40 rows)
Loaded: C1_Lubrication_PM_Plan.csv (41 rows)
Loaded: C1_Mechanical_PM_Plan.csv (43 rows)
Loaded: C4_PM_Plan.csv (60 rows)
Loaded: C5_Lubrication_PM_Plan.csv (47 rows)
Loaded: C5_Mechanical_PM_Plan.csv (60 rows)
Loaded: Duplex_Lubrication_PM_Plan.csv (28 rows)
Loaded: Duplex_Mechanical_PM_Plan.csv (35 rows)
Loaded: G1_PM_Plan.csv (8 rows)
Loaded: Roller_Head_Lubrication_PM_Plan.csv (47 rows)
Loaded: Roller_Head_Mechanical_PM_Plan.csv (60 rows)
Loaded: Triplex_Lubrication_PM_Plan.csv (41 rows)
Loaded: Trip